<a href="https://colab.research.google.com/github/K-107/Ventilator_Pressure_Prediction/blob/main/test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://www.kaggle.com/deepak915/best-mae-score-starter-guide

In [1]:
% cd /content/drive/MyDrive/Ventilator_Pressure_Prediction

/content/drive/MyDrive/Ventilator_Pressure_Prediction


In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [25]:
train_df

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   breath_id  int64  
 2   R          int64  
 3   C          int64  
 4   time_step  float64
 5   u_in       float64
 6   u_out      int64  
 7   pressure   float64
dtypes: float64(3), int64(5)
memory usage: 368.4 MB


In [9]:
train_df.isnull().sum()

id           0
breath_id    0
R            0
C            0
time_step    0
u_in         0
u_out        0
pressure     0
dtype: int64

In [10]:
Y=train_df['pressure']

In [12]:
train_df['u_in'].groupby(train_df['breath_id']).sum()

breath_id
1         811.680524
2         407.499781
3         436.607933
4         480.179073
5         972.724846
             ...    
125740    269.306691
125742    631.366855
125743    188.701654
125745    464.200704
125749    392.912174
Name: u_in, Length: 75450, dtype: float64

In [18]:
train_df['u_in'].groupby(train_df['breath_id']).mean()

breath_id
1         10.146007
2          5.093747
3          5.457599
4          6.002238
5         12.159061
            ...    
125740     3.366334
125742     7.892086
125743     2.358771
125745     5.802509
125749     4.911402
Name: u_in, Length: 75450, dtype: float64

In [32]:
train_df['u_in'].groupby(train_df['breath_id']).cumsum()

0            0.083334
1           18.466375
2           40.975653
3           63.784476
4           89.140326
              ...    
6035995    387.109297
6035996    388.597794
6035997    390.156772
6035998    391.429435
6035999    392.912174
Name: u_in, Length: 6036000, dtype: float64

groupby에서 cumsum은 같은 그룹에서(groupby(train_df['breath_id']))만 누적합을 하는 것이다!!

In [31]:
train_df['u_in'].groupby(train_df['breath_id']).cumsum()[70:81]

70    766.933025
71    771.882629
72    776.840094
73    781.804230
74    786.773959
75    791.748433
76    796.726915
77    801.708762
78    806.693445
79    811.680524
80     12.184338
Name: u_in, dtype: float64

In [30]:
train_df[train_df['breath_id'] == 1].u_in.sum()

811.6805241019604

In [33]:
train_df['u_in_cumsum'] = (train_df['u_in']).groupby(train_df['breath_id']).cumsum()
test_df['u_in_cumsum'] = (test_df['u_in']).groupby(test_df['breath_id']).cumsum()

In [34]:
from sklearn.feature_selection import mutual_info_regression
# Mutual info regression will provide a mesure of dependence between the Features and Traget-variable

xtr=train_df.drop(['pressure','id'],axis=1)[:70000]
# Visualizing the dependence using first 70,000 rows

ytr=train_df['pressure'][:70000]

In [35]:
mi_score=mutual_info_regression(xtr,ytr)
# Providing the features and Target-variable as input

mi_score=pd.Series(mi_score,index=xtr.columns)
#converting the scores to a pandas series and providing the columns as index for the scores

mi_score=mi_score.sort_values(ascending=False)
# Sorting the scores in descending order

mi_score*100
# Getting the percentage-dependece out of 100

time_step      73.004392
u_in           45.271698
breath_id      36.599210
u_out          30.552758
u_in_cumsum    28.981456
R               5.586585
C               5.071199
dtype: float64

In [36]:
from sklearn.preprocessing import StandardScaler
xtr=StandardScaler().fit_transform(train_df.drop(['pressure','R','C','id'],axis=1))
xte=StandardScaler().fit_transform(test_df.drop(['R','C','id'],axis=1))